In [2]:
import os
import platform
import cv2 as cv
import json
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers, models, optimizers
from sys import path
path.insert(0, '..')
import model
import config
import utill
from layer import MobileNetV2

In [3]:
inputs = layers.Input((720, 720, 3))
mdl = MobileNetV2(inputs, 8, 7)

2022-03-08 20:20:23.358742: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-08 20:20:23.358800: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ozmonday): /proc/driver/nvidia/version does not exist
2022-03-08 20:20:23.359174: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
mdl.summary()


Model: "mobilenetv2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 720, 720, 3)]     0         
_________________________________________________________________
conv_block_0 (Conv2D)        (None, 360, 360, 32)      864       
_________________________________________________________________
conv_block_0_bn (BatchNormal (None, 360, 360, 32)      128       
_________________________________________________________________
conv_block_0_relu (ReLU)     (None, 360, 360, 32)      0         
_________________________________________________________________
conv_block_1 (Conv2D)        (None, 360, 360, 32)      1024      
_________________________________________________________________
conv_block_1_bn (BatchNormal (None, 360, 360, 32)      128       
_________________________________________________________________
conv_block_1_relu (ReLU)     (None, 360, 360, 32)      

In [8]:
print(mdl.outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 15, 15, 91), dtype=tf.float32, name=None), name='conv2d/BiasAdd:0', description="created by layer 'conv2d'")
